In [1]:
import os 
l=[]
data_dir='Spine/'
for folder in os.scandir(data_dir):
    
    for entry in os.scandir(data_dir + folder.name): 
        l.append(data_dir + folder.name + '/' + entry.name) 
import random 
random.shuffle(l)

In [1]:
# for i in l:
#     print(i.split('/')[1]) 


In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 224

data_dir = 'Spine/'

X = []
y = []
for folder in os.scandir(data_dir):
    print(folder)

    for entry in os.scandir(data_dir + folder.name):
        img = cv2.imread(data_dir + folder.name + '/' + entry.name, cv2.IMREAD_COLOR)  
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = np.array(img, dtype='float32')
        img = img / 255
        X.append(img)
        if folder.name == 'normal':
            y.append(0)
        elif folder.name == 'osteophenia':
            y.append(1)
        else:
            y.append(2)

X = np.array(X)
y = np.array(y)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

batch_size = 32
num_batches = 90  

augmented_images = []
augmented_labels = []
for _ in range(num_batches):
    X_batch, y_batch = next(datagen.flow(X, y, batch_size=batch_size, shuffle=False))
    augmented_images.append(X_batch)
    augmented_labels.append(y_batch)

X_augmented = np.concatenate(augmented_images, axis=0)
y_augmented = np.concatenate(augmented_labels, axis=0)



<DirEntry 'normal'>
<DirEntry 'osteophenia'>
<DirEntry 'osteoporosis'>


In [12]:
X_augu = np.concatenate((X, X_augmented), axis=0)
y_augu=np.concatenate((y, y_augmented), axis=0)


In [ ]:
d={0:0,1:0,2:0}
for i in y_augu:
    i=np.argmax(i)
    d[i]+=1 
d

In [15]:
print(y_aug)

[1 1 0 ... 2 0 1]


In [5]:
y

array([1, 1, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 1, 0, 2, 0, 2,
       1, 1, 1, 1, 2, 0, 2, 2, 0, 1, 0, 2, 2, 2, 0, 1, 1, 2, 2, 2, 0, 0,
       2, 1, 0, 0, 1, 1, 2, 2, 1, 2, 1, 2, 1, 0, 2, 2, 1, 0, 0, 2, 2, 2,
       1, 0, 2, 2, 0, 2, 1, 1, 1, 2, 0, 1, 1, 2, 2, 0, 0, 0, 1, 0, 2, 0,
       1, 2, 0, 2, 0, 1, 1, 0, 1, 1, 2, 0, 2, 1, 2, 1, 0, 0, 1, 0, 0, 0,
       2, 0, 1, 1, 0, 1, 1, 0, 2, 1, 0, 1, 2, 1, 0, 0, 1, 1, 0, 0, 1, 2,
       1, 2, 1, 0, 0, 0, 1, 1, 2, 0, 2, 0, 0, 1, 1, 2, 2, 1, 1, 0, 1, 0,
       0, 2, 1, 1, 2, 1, 0, 1, 0, 0, 1, 1, 0, 2, 0, 2, 2, 1, 2, 0])

In [6]:
from keras.models import load_model
m1=load_model('Spine_DNET_with_aug.h5')
m2=load_model('Ravispine_MobileNetv2.h5')
m3=load_model('Spine_InceptionV3_with_aug.h5')
m4=load_model('Spine_VGG19_with_aug.h5')
m5=load_model('rahul_spine_resnet_with_aug.h5')

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_augu, y_augu, test_size=0.2, shuffle=True, stratify=y_augu, random_state=123)

In [40]:
d={0:0,1:0,2:0}
for i in y_train:
    d[i]+=1 
d

{0: 2352, 1: 48, 2: 43}

In [41]:
d={0:0,1:0,2:0}
for i in y_test:
    d[i]+=1 
d

{0: 588, 1: 12, 2: 11}

In [4]:
loss,accuracy=m1.evaluate(X_test,y_test)
print(accuracy)

20/20 [==============================] - 62s 3s/step - loss: 0.7885 - accuracy: 0.9624
0.9623568058013916


In [5]:
loss1,accuracy1=m2.evaluate(X_test,y_test)
print(accuracy1)

20/20 [==============================] - 17s 755ms/step - loss: 0.0776 - accuracy: 0.9607
0.9607201218605042


In [6]:
loss2,accuracy2=m3.evaluate(X_test,y_test)
print(accuracy2)

20/20 [==============================] - 25s 1s/step - loss: 0.1334 - accuracy: 0.9624
0.9623568058013916


In [7]:
loss3,accuracy3=m4.evaluate(X_test,y_test)
print(accuracy3)

20/20 [==============================] - 117s 6s/step - loss: 0.0877 - accuracy: 0.9624
0.9623568058013916


In [10]:
loss4,accuracy4=m5.evaluate(X_test,y_test)
print(accuracy4)

20/20 [==============================] - 49s 2s/step - loss: 0.6358 - accuracy: 0.9624
0.9623568058013916


In [20]:
y_pred=m1.predict(X_test)

20/20 [==============================] - 50s 2s/step


In [21]:
y_pred.shape

(611, 3)

In [33]:
l=[]
for i in y_pred:
    l.append(np.argmax(i))
y_predd=np.array(l)

In [34]:
y_predd.shape

(611,)

In [35]:
y_test.shape

(611,)

In [36]:
from sklearn.metrics import classification_report 
report=classification_report(y_predd,y_test)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.96      0.98       611
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0

    accuracy                           0.96       611
   macro avg       0.33      0.32      0.33       611
weighted avg       1.00      0.96      0.98       611



D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import numpy as np
# Load augmented_images array
augmented_images = np.load('Augmented_images.npy')

# Load augmented_labels array
augmented_labels = np.load('Augmented_labels.npy')


In [15]:
import numpy as np
from keras.models import load_model
from sklearn.metrics import accuracy_score

# Load the models
m1 = load_model('Spine_DNET_with_aug.h5')
m2 = load_model('Ravispine_MobileNetv2.h5')
m3 = load_model('Spine_InceptionV3_with_aug.h5')
m4 = load_model('Spine_VGG19_with_aug.h5')
m5 = load_model('rahul_spine_resnet_with_aug.h5')

# Make predictions using each model
preds_m1 = np.argmax(m1.predict(X_test), axis=1)
preds_m2 = np.argmax(m2.predict(X_test), axis=1)
preds_m3 = np.argmax(m3.predict(X_test), axis=1)
preds_m4 = np.argmax(m4.predict(X_test), axis=1)
preds_m5 = np.argmax(m5.predict(X_test), axis=1)

# Calculate accuracy for each class for each model
class_wise_accuracy_m1 = []
class_wise_accuracy_m2 = []
class_wise_accuracy_m3 = []
class_wise_accuracy_m4 = []
class_wise_accuracy_m5 = []

for cls in range(3):  # Assuming num_classes is the number of classes in your dataset
    idx = np.where(y_test == cls)[0]
    class_preds_m1 = preds_m1[idx]
    class_preds_m2 = preds_m2[idx]
    class_preds_m3 = preds_m3[idx]
    class_preds_m4 = preds_m4[idx]
    class_preds_m5 = preds_m5[idx]

    class_true_labels = y_test[idx]

    class_wise_accuracy_m1.append(accuracy_score(class_true_labels, class_preds_m1))
    class_wise_accuracy_m2.append(accuracy_score(class_true_labels, class_preds_m2))
    class_wise_accuracy_m3.append(accuracy_score(class_true_labels, class_preds_m3))
    class_wise_accuracy_m4.append(accuracy_score(class_true_labels, class_preds_m4))
    class_wise_accuracy_m5.append(accuracy_score(class_true_labels, class_preds_m5))

# Print the class-wise accuracy for each model
print("Model 1 Class-wise Accuracy:", class_wise_accuracy_m1)
print("Model 2 Class-wise Accuracy:", class_wise_accuracy_m2)
print("Model 3 Class-wise Accuracy:", class_wise_accuracy_m3)
print("Model 4 Class-wise Accuracy:", class_wise_accuracy_m4)
print("Model 5 Class-wise Accuracy:", class_wise_accuracy_m5)


20/20 [==============================] - 44s 2s/step
Model 1 Class-wise Accuracy: [1.0, 0.0, 0.0]
Model 2 Class-wise Accuracy: [0.9812925170068028, 0.8333333333333334, 0.0]
Model 3 Class-wise Accuracy: [1.0, 0.0, 0.0]
Model 4 Class-wise Accuracy: [1.0, 0.0, 0.0]
Model 5 Class-wise Accuracy: [1.0, 0.0, 0.0]


In [1]:
# import numpy as np
from keras.models import load_model
# from sklearn.metrics import accuracy_score

# Load the models
m1 = load_model('Spine_VGG19_with_aug.h5')

In [2]:
from tensorflow.keras.utils import model_to_dot
from IPython.display import SVG
import pydotplus
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



# Convert the Keras model to a dot format
dot = model_to_dot(m1, show_shapes=True, show_layer_names=True)

# Convert dot to svg
svg = dot.create_svg()

# Display the SVG
SVG(svg)


RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xe

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xe

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xe

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception

In [1]:
conda install python-graphviz


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: D:\anaconda

  added / updated specs:
    - python-graphviz


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2021.11  |           py39_0           7 KB
    anaconda-custom            |           py39_2          13 KB
    ca-certificates-2024.3.11  |       haa95532_0         128 KB
    certifi-2024.2.2           |   py39haa95532_0         160 KB
    fftw-3.3.9                 |       h2bbff1b_2         1.0 MB
    graphviz-2.38              |       hfd603c8_2        29.3 MB
    h5py-3.7.0                 |   py39h3de5c98_0         823 KB
    openssl-1.1.1w             |       h2bbff1b_0         5.5 MB
    pillow-9.3.0               |   py39hdc2b20a_1 


The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2021.11=py39_0
  - defaults/win-64::anaconda-navigator==2.1.1=py39_0
  - defaults/noarch::anaconda-project==0.10.1=pyhd3eb1b0_0
  - defaults/win-64::astropy==4.3.1=py39hc7d831d_0
  - defaults/win-64::bkcharts==0.2=py39haa95532_0
  - defaults/noarch::black==19.10b0=py_0
  - defaults/win-64::bokeh==2.4.1=py39haa95532_0
  - defaults/win-64::bottleneck==1.3.2=py39h7cc1a96_1
  - defaults/win-64::conda==4.10.3=py39haa95532_0
  - defaults/win-64::conda-build==3.21.6=py39haa95532_0
  - defaults/win-64::conda-package-handling==1.7.3=py39h8cc25b3_1
  - defaults/noarch::conda-token==0.3.0=pyhd3eb1b0_0
  - defaults/noarch::conda-verify==3.4.2=py_1
  - defaults/win-64::daal4py==2021.3.0=py39h757b272_0
  - defaults/noarch::dask==2021.10.0=pyhd3eb1b0_0
  - defaults/win-64::imagecodecs==2021.8.26=py39ha1f97ea_0
  - defaults/noarch::imageio==2.9


graphviz-2.38        | 29.3 MB   | 2          |   2% 
graphviz-2.38        | 29.3 MB   | 3          |   4% 
graphviz-2.38        | 29.3 MB   | 5          |   6% 
graphviz-2.38        | 29.3 MB   | 7          |   8% 
graphviz-2.38        | 29.3 MB   | #          |  10% 
graphviz-2.38        | 29.3 MB   | #2         |  13% 
graphviz-2.38        | 29.3 MB   | #4         |  15% 
graphviz-2.38        | 29.3 MB   | #6         |  16% 
graphviz-2.38        | 29.3 MB   | #8         |  19% 
graphviz-2.38        | 29.3 MB   | ##1        |  21% 
graphviz-2.38        | 29.3 MB   | ##3        |  24% 
graphviz-2.38        | 29.3 MB   | ##8        |  28% 
graphviz-2.38        | 29.3 MB   | ###        |  31% 
graphviz-2.38        | 29.3 MB   | ###3       |  33% 
graphviz-2.38        | 29.3 MB   | ###5       |  35% 
graphviz-2.38        | 29.3 MB   | ###7       |  37% 
graphviz-2.38        | 29.3 MB   | ###9       |  40% 
graphviz-2.38        | 29.3 MB   | ####1      |  42% 
graphviz-2.38        | 29.3



The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2021.11=py39_0
  - defaults/win-64::anaconda-navigator==2.1.1=py39_0
  - defaults/noarch::anaconda-project==0.10.1=pyhd3eb1b0_0
  - defaults/win-64::astropy==4.3.1=py39hc7d831d_0
  - defaults/win-64::bkcharts==0.2=py39haa95532_0
  - defaults/noarch::black==19.10b0=py_0
  - defaults/win-64::bokeh==2.4.1=py39haa95532_0
  - defaults/win-64::bottleneck==1.3.2=py39h7cc1a96_1
  - defaults/win-64::conda==4.10.3=py39haa95532_0
  - defaults/win-64::conda-build==3.21.6=py39haa95532_0
  - defaults/win-64::conda-package-handling==1.7.3=py39h8cc25b3_1
  - defaults/noarch::conda-token==0.3.0=pyhd3eb1b0_0
  - defaults/noarch::conda-verify==3.4.2=py_1
  - defaults/win-64::daal4py==2021.3.0=py39h757b272_0
  - defaults/noarch::dask==2021.10.0=pyhd3eb1b0_0
  - defaults/win-64::imagecodecs==2021.8.26=py39ha1f97ea_0
  - defaults/noarch::imageio==2.


scipy-1.7.3          | 13.9 MB   | ##7        |  28% 
scipy-1.7.3          | 13.9 MB   | ###1       |  31% 
scipy-1.7.3          | 13.9 MB   | ###4       |  35% 
scipy-1.7.3          | 13.9 MB   | ###9       |  39% 
scipy-1.7.3          | 13.9 MB   | ####4      |  44% 
scipy-1.7.3          | 13.9 MB   | ####8      |  49% 
scipy-1.7.3          | 13.9 MB   | #####2     |  53% 
scipy-1.7.3          | 13.9 MB   | #####6     |  56% 
scipy-1.7.3          | 13.9 MB   | ######     |  61% 
scipy-1.7.3          | 13.9 MB   | ######4    |  64% 
scipy-1.7.3          | 13.9 MB   | ######8    |  68% 
scipy-1.7.3          | 13.9 MB   | #######2   |  72% 
scipy-1.7.3          | 13.9 MB   | #######6   |  77% 
scipy-1.7.3          | 13.9 MB   | ########   |  80% 
scipy-1.7.3          | 13.9 MB   | ########3  |  84% 
scipy-1.7.3          | 13.9 MB   | ########7  |  88% 
scipy-1.7.3          | 13.9 MB   | #########1 |  92% 
scipy-1.7.3          | 13.9 MB   | #########6 |  96% 
scipy-1.7.3          | 13.9